In [20]:
from datasets import load_dataset
from model import NERClassifier
from preprocess_dataset import NERDataset 
from trainner import Trainner
from transformers import BertTokenizerFast
from preprocess_dataset import remove_empty_entries

import numpy as np
import torch

## Checkpoints to be used

In [15]:
pt_base_cased    = 'neuralmind/bert-base-portuguese-cased'
pt_large_cased   = 'neuralmind/bert-large-portuguese-cased'
pt_base_uncased  = 'neuralmind/bert-base-portuguese-uncased'
pt_large_uncased = 'neuralmind/bert-large-portuguese-uncased'
en_base_uncased = 'bert-base-uncased'
en_large_uncased = 'bert-large-uncased'
en_base_cased = 'bert-base-cased'
en_large_cased = 'bert-large-cased'

pt = [pt_base_cased, pt_large_cased, pt_base_uncased, pt_large_uncased]
en = [en_base_cased, en_large_cased, en_base_uncased, en_large_uncased]
cased = [pt_base_case, en_base_case, pt_large_case, en_large_case]
uncased = [en_base_uncased, en_large_uncased, pt_base_uncased, pt_large_uncased] 

In [31]:
models = [pt, en, cased, uncased]

In [34]:
flat_models = [check for checkpoints in models for check in checkpoints]

In [35]:
flat_models

['neuralmind/bert-base-portuguese-cased',
 'neuralmind/bert-large-portuguese-cased',
 'neuralmind/bert-base-portuguese-uncased',
 'neuralmind/bert-large-portuguese-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-uncased',
 'bert-large-uncased',
 'neuralmind/bert-base-portuguese-cased',
 'bert-base-cased',
 'neuralmind/bert-large-portuguese-cased',
 'bert-large-cased',
 'bert-base-uncased',
 'bert-large-uncased',
 'neuralmind/bert-base-portuguese-uncased',
 'neuralmind/bert-large-portuguese-uncased',
 'bert-base-uncased',
 'bert-large-uncased',
 'neuralmind/bert-base-portuguese-uncased',
 'neuralmind/bert-large-portuguese-uncased']

## Dataset

In [36]:
data = "lener_br"
dataset = load_dataset(data)
dataset = remove_empty_entries(dataset)

Reusing dataset lener_br (/home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382)
Loading cached processed dataset at /home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-5e59bc59f25f3d7f.arrow
Loading cached processed dataset at /home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-8d0457760cd67ee6.arrow
Loading cached processed dataset at /home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382/cache-74e841c1c151996a.arrow


## Hyperparameters

In [40]:
MAX_LEN = 128
LEARNING_RATE=3e-4
n_labels = 13
BATCH_SIZE=8
shuffle=True
NUM_EPOCHS=1
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")